In [97]:
import sys
import os
import pandas as pd

import llm_run_functions as lf

In [98]:
def silito():
    df = pd.read_csv("silito/silito_data/memos-1.csv")
    system_prompt_file='prompt_two_examples.txt'

    system_prompt = lf.read_system_prompt(system_prompt_file)

    batch_prompts = []
    batch_metadata = []
    results = []

    def generate_silito_prompt(row, system_prompt):
        user_prompt = f"""Text Requiring Open Codes:
                    Input:
                    Interviewer: AI Research
                    Answer: {row['answer']}
                    JSON Output:
                    """
        return system_prompt + '\n' + user_prompt

    for index, row in df.iterrows():
        full_prompt = generate_silito_prompt(row, system_prompt)
        batch_prompts.append(full_prompt)
        this_metadata = {'answer': row['answer'], 'human_code': row['Code'], 'model': 'Llama'}
        batch_metadata.append(this_metadata)
        res = lf.llama3(full_prompt)
        results.append(res)

    new_rows = []
    bad_rows = []
    for metadata, result in zip(batch_metadata, results):
        # print(result)
        if result is not None:
            # print(result)
            parsed_json = lf.parse_result(result)
            if parsed_json is not None:
                metadata['json'] = parsed_json
                new_rows.append(metadata)
            else:
                # print("couldn't parse this: ", metadata[language_in])
                metadata['result'] = result
                bad_rows.append(metadata)

    output_csv_path = 'silito/silito_eye_test/g_memo-1_good.csv'
    bad_csv_path = 'silito/silito_eye_test/g_memo-1_bad.csv'
    if new_rows:
        print(f"Writing {len(new_rows)} rows to csv")
        # print(f"New ids completed: {[row['id'] for row in new_rows]}")
        new_df = pd.DataFrame(new_rows)
        if os.path.exists(output_csv_path):
            new_df.to_csv(output_csv_path, mode='a', header=False, index=False)
        else:
            new_df.to_csv(output_csv_path, index=False)

    if bad_rows:
        bad_df = pd.DataFrame(bad_rows)
        if os.path.exists(bad_csv_path):
            bad_df.to_csv(bad_csv_path, mode='a', header=False, index=False)
        else:
            bad_df.to_csv(bad_csv_path, index=False)

silito()

JSON decoding error: Invalid \escape: line 2 column 32 (char 154)
Writing 18 rows to csv


In [99]:
import ast

df = pd.read_csv('silito/silito_eye_test/g_memo-1_good.csv')
df['json'] = df['json'].apply(lambda x: ast.literal_eval(x))
df = df.explode(column='json')
df['code'] = df['json'].apply(lambda x: x['Code'])
df['text'] = df['json'].apply(lambda x: x['Original_Text'])

In [100]:
df

,answer,human_code,model,json,code,text
0,"- From a lesson learned ""you know that bad dat...","['Similarity, going beyond exact scenario.']",Llama,{'Original_Text': 'you know that bad data left...,Risk of Bad Data,you know that bad data left on a system is dan...
0,"- From a lesson learned ""you know that bad dat...","['Similarity, going beyond exact scenario.']",Llama,{'Original_Text': 'find every place where we h...,Data Evaluation Failure,find every place where we have a data evaluati...
0,"- From a lesson learned ""you know that bad dat...","['Similarity, going beyond exact scenario.']",Llama,{'Original_Text': 'make certain the data is re...,Data Removal,make certain the data is removed.
1,- LLs are covered by a single AI OR single LL ...,['Relation of AIs to LLs'],Llama,{'Original_Text': 'LLs are covered by a single...,LL Coverage by Single AI,LLs are covered by a single AI
1,- LLs are covered by a single AI OR single LL ...,['Relation of AIs to LLs'],Llama,{'Original_Text': 'single LL is covered by sev...,LL Coverage by Multiple AIs,single LL is covered by several AIs
...,...,...,...,...,...,...
15,- > because every change is a risk.,['Changing something adds risk of future incid...,Llama,{'Original_Text': 'because every change is a r...,Change as Risk,because every change is a risk.
16,- We do know how to engineer a reliable system...,"[""Intrinsic need for FDSE if we don't want to ...",Llama,{'Original_Text': 'We do know how to engineer ...,Knowledge vs. Implementation,"We do know how to engineer a reliable system, ..."
16,- We do know how to engineer a reliable system...,"[""Intrinsic need for FDSE if we don't want to ...",Llama,{'Original_Text': 'This mindset directly leads...,Mindset Drives Need for FDSE,This mindset directly leads to needing to have...
17,- Things work perfectly in their isolated litt...,['Reason for change'],Llama,{'Original_Text': 'Things work perfectly in th...,Isolated Functionality,Things work perfectly in their isolated little...


In [101]:
df

,answer,human_code,model,json,code,text
0,"- From a lesson learned ""you know that bad dat...","['Similarity, going beyond exact scenario.']",Llama,{'Original_Text': 'you know that bad data left...,Risk of Bad Data,you know that bad data left on a system is dan...
0,"- From a lesson learned ""you know that bad dat...","['Similarity, going beyond exact scenario.']",Llama,{'Original_Text': 'find every place where we h...,Data Evaluation Failure,find every place where we have a data evaluati...
0,"- From a lesson learned ""you know that bad dat...","['Similarity, going beyond exact scenario.']",Llama,{'Original_Text': 'make certain the data is re...,Data Removal,make certain the data is removed.
1,- LLs are covered by a single AI OR single LL ...,['Relation of AIs to LLs'],Llama,{'Original_Text': 'LLs are covered by a single...,LL Coverage by Single AI,LLs are covered by a single AI
1,- LLs are covered by a single AI OR single LL ...,['Relation of AIs to LLs'],Llama,{'Original_Text': 'single LL is covered by sev...,LL Coverage by Multiple AIs,single LL is covered by several AIs
...,...,...,...,...,...,...
15,- > because every change is a risk.,['Changing something adds risk of future incid...,Llama,{'Original_Text': 'because every change is a r...,Change as Risk,because every change is a risk.
16,- We do know how to engineer a reliable system...,"[""Intrinsic need for FDSE if we don't want to ...",Llama,{'Original_Text': 'We do know how to engineer ...,Knowledge vs. Implementation,"We do know how to engineer a reliable system, ..."
16,- We do know how to engineer a reliable system...,"[""Intrinsic need for FDSE if we don't want to ...",Llama,{'Original_Text': 'This mindset directly leads...,Mindset Drives Need for FDSE,This mindset directly leads to needing to have...
17,- Things work perfectly in their isolated litt...,['Reason for change'],Llama,{'Original_Text': 'Things work perfectly in th...,Isolated Functionality,Things work perfectly in their isolated little...


# Making Map

In [102]:
code_map = {}

# Iterate through the dataframe to build the map
for idx, row in df.iterrows():
    for code_str in row['code']:  # Iterate through the list in the 'code' column
        code_map[code_str] = row['answer']

# Clustering

In [103]:
all_codes = [string for string in df['code']]
data = {
    'Group': 'One',
    'codes': [all_codes]
}
all_codes = pd.DataFrame(data)
all_codes

,Group,codes
0,One,"[Risk of Bad Data, Data Evaluation Failure, Da..."


In [104]:
from sklearn.cluster import HDBSCAN
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import silhouette_score
from sklearn.metrics import silhouette_samples
from sklearn.preprocessing import normalize
import umap.umap_ as umap
from sentence_transformers.util import cos_sim
from sentence_transformers import SentenceTransformer
import numpy as np
from collections import defaultdict


model_ckpt = 'sentence-transformers/distiluse-base-multilingual-cased-v2'
sim_model = SentenceTransformer(model_ckpt)

def hdb_clustering(all_codes, n_neighbors = 5, n_components = 20, min_cluster_size = 5, min_sample_size =5):
    umap_model = umap.UMAP(n_neighbors=n_neighbors, n_components=n_components, min_dist=0.0, metric='cosine', random_state=42)
    hdb = HDBSCAN(min_cluster_size=min_cluster_size, min_samples=min_sample_size)

    all_codes['clusters'] = None
    new_rows = []
    silhouette_scores = []

    for index, row in all_codes.iterrows():
        X = sim_model.encode(row['codes'])

        try:
            X = umap_model.fit_transform(X)
        except Exception as e:
            return None
        X_norm = normalize(X) # L2 Normalized Euclidean == cosine similarity
        hdb.fit(X_norm)
        hdb_label_ids = set(hdb.labels_)
        hdb_centroids = np.array([
            X[hdb.labels_ == i].mean(axis=0)
            for i in hdb_label_ids
        ])
        hdb_centers = np.array([
            np.argmax(np.dot(X_norm, centroid.T))
            for centroid in hdb_centroids
        ])

        # Silhouette Score
        sample_silhouette_values = silhouette_samples(X_norm, hdb.labels_)

        # Exclude noise points (-1 labels)
        non_noise_mask = hdb.labels_ != -1
        filtered_sample_silhouette_values = sample_silhouette_values[non_noise_mask]
        filtered_labels = hdb.labels_[non_noise_mask]

        # Compute the average silhouette score excluding noise
        silhouette_avg = np.mean(filtered_sample_silhouette_values)
        silhouette_scores.append(silhouette_avg)

        hdb_dict = defaultdict(list)
        for label, sent in zip(hdb.labels_, row['codes']):
            hdb_dict[label].append(sent)

        # Create new rows using the dictionary
        for label, cluster_sentences in hdb_dict.items():
            new_row = row.copy()
            new_row['clusters_size'] = len(cluster_sentences)
            new_row['clusters'] = cluster_sentences
            new_row['cluster_id'] = label
            new_row['center'] = row['codes'][hdb_centers[label]]
            new_row['silhouette_avg'] = silhouette_avg
            new_row['silhouette'] = sample_silhouette_values[label]
            new_rows.append(new_row)

    clustered_df = pd.DataFrame(new_rows)
    clustered_df.reset_index(drop=True, inplace=True)

    return clustered_df, np.mean(silhouette_scores)

In [105]:
clustered_df, _ = hdb_clustering(all_codes, 5, 15, 5, 5)

/Users/ethan/PycharmProjects/axial-coding/.venv/lib/python3.9/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


In [106]:
clustered_df

,Group,codes,clusters,clusters_size,cluster_id,center,silhouette_avg,silhouette
0,One,"[Risk of Bad Data, Data Evaluation Failure, Da...","[Risk of Bad Data, Data Evaluation Failure, Da...",19,3,Subjectivity in Evaluation,0.588289,0.713318
1,One,"[Risk of Bad Data, Data Evaluation Failure, Da...","[LL Coverage by Single AI, LL Coverage by Mult...",16,2,LL Coverage by Single AI,0.588289,0.646613
2,One,"[Risk of Bad Data, Data Evaluation Failure, Da...","[Process Deviation, Process Over Product, Leng...",6,-1,Process Over Product,0.588289,0.230003
3,One,"[Risk of Bad Data, Data Evaluation Failure, Da...","[Quality Control Role, Verification of Complet...",6,0,Independent Verification,0.588289,0.668809
4,One,"[Risk of Bad Data, Data Evaluation Failure, Da...","[Application to Deployment, Decision Weighing,...",7,1,Cost of Implementation,0.588289,0.680608
5,One,"[Risk of Bad Data, Data Evaluation Failure, Da...","[Scope of Action Items, Time Constraint on Act...",14,4,Potential Damage,0.588289,0.662800


# Axial Code Generation

In [107]:
clustered_df['prompt'] = 'AICode'

In [108]:
def generate_prompt(row, system_prompt):
    user_prompt = f"""Open Codes requiring Axial Codes:
                Input:
                Open Codes: {row['clusters']}
                SentenceBERT center: {row['center']}
                JSON Output:
                """
    return system_prompt + '\n' + user_prompt

In [109]:
def run_llm(df, batch_size = 10):
    prompt_file = 'silito/axial_prompt.txt'
    system_prompt = lf.read_system_prompt(prompt_file)
    output_csv_path = 'silito/output_axials/Good_gemma3.csv'
    bad_csv_path = 'silito/output_axials/Bad_gemma3.csv'

    if os.path.exists(output_csv_path):
        comp_df = pd.read_csv(output_csv_path)
        df = df[~df['cluster_id'] in set(comp_df['cluster_id'])]

    i = 0
    tot = len(df) // batch_size
    for batch in lf.batch_iterator(df.iterrows(), batch_size):
        i += 1
        if i % 4 == 0:
            print(f"{i} : {tot}")

        batch_prompts = []
        batch_metadata = []
        results = []
        for index, row in batch:
            full_prompt = generate_prompt(row, system_prompt)
            batch_prompts.append(full_prompt)
            this_metadata = {'id': row['cluster_id'], 'model': 'Gemma'}
            batch_metadata.append(this_metadata)
            res = lf.llama3(full_prompt)
            results.append(res)

        new_rows = []
        bad_rows = []
        for metadata, result in zip(batch_metadata, results):
            # print(result)
            if result is not None:
                # print(result)
                parsed_json = lf.parse_result(result)
                if parsed_json is not None:
                    metadata['codes'] = parsed_json
                    new_rows.append(metadata)
                else:
                    # print("couldn't parse this: ", metadata[language_in])
                    metadata['result'] = result
                    bad_rows.append(metadata)

        if new_rows:
            print(f"Writing {len(new_rows)} rows to csv")
            # print(f"New ids completed: {[row['id'] for row in new_rows]}")
            new_df = pd.DataFrame(new_rows)
            if os.path.exists(output_csv_path):
                new_df.to_csv(output_csv_path, mode='a', header=False, index=False)
            else:
                new_df.to_csv(output_csv_path, index=False)

        if bad_rows:
            bad_df = pd.DataFrame(bad_rows)
            if os.path.exists(bad_csv_path):
                bad_df.to_csv(bad_csv_path, mode='a', header=False, index=False)
            else:
                bad_df.to_csv(bad_csv_path, index=False)

run_llm(clustered_df)

In [116]:
df_axial = pd.read_csv('silito/output_axials/Bad_gemma3.csv')
df_axial['Axial_Code'] = df_axial['result'].apply(lambda x: ast.literal_eval(x.strip("`json"))['Axial_Category'])
df_axial

,id,model,result,Axial_Code
0,3,Gemma,"```json\n{\n ""Axial_Category"": ""Managing Unce...",Managing Uncertainty in Data-Driven Change
1,2,Gemma,"```json\n{\n ""Axial_Category"": ""AI as a tool ...",AI as a tool for scalable problem solving
2,-1,Gemma,"```json\n{\n ""Axial_Category"": ""Prioritizing ...",Prioritizing learning over efficient processes
3,0,Gemma,"```json\n{\n ""Axial_Category"": ""Ensuring proc...",Ensuring procedural correctness
4,1,Gemma,"```json\n{\n ""Axial_Category"": ""Balancing inv...",Balancing investment with practical application
5,4,Gemma,"```json\n{\n ""Axial_Category"": ""Proactive Ris...",Proactive Risk Mitigation


In [117]:
df_full = pd.merge(clustered_df, df_axial, how='left', left_on='cluster_id', right_on='id')
df_full

,Group,codes,clusters,clusters_size,cluster_id,center,silhouette_avg,silhouette,prompt,id,model,result,Axial_Code
0,One,"[Risk of Bad Data, Data Evaluation Failure, Da...","[Risk of Bad Data, Data Evaluation Failure, Da...",19,3,Subjectivity in Evaluation,0.588289,0.713318,AICode,3,Gemma,"```json\n{\n ""Axial_Category"": ""Managing Unce...",Managing Uncertainty in Data-Driven Change
1,One,"[Risk of Bad Data, Data Evaluation Failure, Da...","[LL Coverage by Single AI, LL Coverage by Mult...",16,2,LL Coverage by Single AI,0.588289,0.646613,AICode,2,Gemma,"```json\n{\n ""Axial_Category"": ""AI as a tool ...",AI as a tool for scalable problem solving
2,One,"[Risk of Bad Data, Data Evaluation Failure, Da...","[Process Deviation, Process Over Product, Leng...",6,-1,Process Over Product,0.588289,0.230003,AICode,-1,Gemma,"```json\n{\n ""Axial_Category"": ""Prioritizing ...",Prioritizing learning over efficient processes
3,One,"[Risk of Bad Data, Data Evaluation Failure, Da...","[Quality Control Role, Verification of Complet...",6,0,Independent Verification,0.588289,0.668809,AICode,0,Gemma,"```json\n{\n ""Axial_Category"": ""Ensuring proc...",Ensuring procedural correctness
4,One,"[Risk of Bad Data, Data Evaluation Failure, Da...","[Application to Deployment, Decision Weighing,...",7,1,Cost of Implementation,0.588289,0.680608,AICode,1,Gemma,"```json\n{\n ""Axial_Category"": ""Balancing inv...",Balancing investment with practical application
5,One,"[Risk of Bad Data, Data Evaluation Failure, Da...","[Scope of Action Items, Time Constraint on Act...",14,4,Potential Damage,0.588289,0.662800,AICode,4,Gemma,"```json\n{\n ""Axial_Category"": ""Proactive Ris...",Proactive Risk Mitigation


# Expand

In [118]:
# Create a mapping from open codes to axial codes
code_to_axial_map = {}

# Loop through df_full and create a mapping for each code in 'clusters' to 'axial_code'
for _, row in df_full.iterrows():
    for cluster_code in row['clusters']:  # Iterate through each open code in 'clusters'
        code_to_axial_map[cluster_code] = row['Axial_Code']

# Now, map the 'code' in df_exploded to the corresponding 'axial_code' in the new column
df['Axial_Code'] = df['code'].map(code_to_axial_map)


In [119]:
df
# df_exploded[['human_code', 'Axial_Code']]

,answer,human_code,model,json,code,text,Axial_Code
0,"- From a lesson learned ""you know that bad dat...","['Similarity, going beyond exact scenario.']",Llama,{'Original_Text': 'you know that bad data left...,Risk of Bad Data,you know that bad data left on a system is dan...,Managing Uncertainty in Data-Driven Change
0,"- From a lesson learned ""you know that bad dat...","['Similarity, going beyond exact scenario.']",Llama,{'Original_Text': 'find every place where we h...,Data Evaluation Failure,find every place where we have a data evaluati...,Managing Uncertainty in Data-Driven Change
0,"- From a lesson learned ""you know that bad dat...","['Similarity, going beyond exact scenario.']",Llama,{'Original_Text': 'make certain the data is re...,Data Removal,make certain the data is removed.,Managing Uncertainty in Data-Driven Change
1,- LLs are covered by a single AI OR single LL ...,['Relation of AIs to LLs'],Llama,{'Original_Text': 'LLs are covered by a single...,LL Coverage by Single AI,LLs are covered by a single AI,AI as a tool for scalable problem solving
1,- LLs are covered by a single AI OR single LL ...,['Relation of AIs to LLs'],Llama,{'Original_Text': 'single LL is covered by sev...,LL Coverage by Multiple AIs,single LL is covered by several AIs,AI as a tool for scalable problem solving
...,...,...,...,...,...,...,...
15,- > because every change is a risk.,['Changing something adds risk of future incid...,Llama,{'Original_Text': 'because every change is a r...,Change as Risk,because every change is a risk.,Managing Uncertainty in Data-Driven Change
16,- We do know how to engineer a reliable system...,"[""Intrinsic need for FDSE if we don't want to ...",Llama,{'Original_Text': 'We do know how to engineer ...,Knowledge vs. Implementation,"We do know how to engineer a reliable system, ...",Balancing investment with practical application
16,- We do know how to engineer a reliable system...,"[""Intrinsic need for FDSE if we don't want to ...",Llama,{'Original_Text': 'This mindset directly leads...,Mindset Drives Need for FDSE,This mindset directly leads to needing to have...,AI as a tool for scalable problem solving
17,- Things work perfectly in their isolated litt...,['Reason for change'],Llama,{'Original_Text': 'Things work perfectly in th...,Isolated Functionality,Things work perfectly in their isolated little...,Ensuring procedural correctness


In [120]:
df.to_csv('silito/combined_out.csv')